In [20]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [21]:
# Download and read the `spacex_launch_geo.csv`
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

spacex_df = pd.read_csv(URL)


In [22]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [23]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [24]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [29]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

for index, site in launch_sites_df.iterrows():
    site_map.add_child(
        folium.Circle(
            location=[site['Lat'], site['Long']],
            radius=1000,  # specify a radius in meters
            color='#000000',  # Circle color
            fill=True,
            fill_color='#000000',
            popup=site['Launch Site']
        ).add_child(folium.Popup(site['Launch Site']))
    )

# Display the map
site_map

In [32]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [33]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Use the MousePosition tool on the map to find the coordinates of a point on the closest coastline to a launch site. Once you have the coordinates, replace the placeholder values in the next code cell with the actual latitude and longitude.

In [35]:
# Replace with the coordinates obtained from the MousePosition tool
coastline_lat = 28.56367  # Example coastline latitude (replace with your value)
coastline_lon = -80.56794 # Example coastline longitude (replace with your value)

# Replace with the latitude and longitude of the launch site you want to measure the distance from
launch_site_lat = 28.562302 # Example launch site latitude (replace with your value)
launch_site_lon = -80.577356 # Example launch site longitude (replace with your value)


if coastline_lat is not None and coastline_lon is not None and launch_site_lat is not None and launch_site_lon is not None:
    distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
    print(f"Distance to coastline: {distance_coastline:.2f} km")

    # Create and add a marker to the map to visualize the distance
    distance_marker = folium.Marker(
       [coastline_lat, coastline_lon],
       icon=DivIcon(
           icon_size=(20,20),
           icon_anchor=(0,0),
           html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} km".format(distance_coastline),
           )
        )
    site_map.add_child(distance_marker)

    # Draw a PolyLine between a launch site to the selected coastline point.
    # Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
    coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
    lines=folium.PolyLine(locations=coordinates, weight=1)
    site_map.add_child(lines)


# Display the map (you may need to re-display the map cell to see the new marker)
site_map

Distance to coastline: 0.93 km
